Calculate $ Z $ and $ V $ for sulfur hexafluoride at $ 75^\circ\text{C} $ and $ 15 $ bar by the following equations:

For sulfur hexafluoride, $ T_c = 318.7 \, \text{K} $, $ P_c = 37.6 \, \text{bar} $, $ V_c = 198 \, \text{cm}^3 \cdot \text{mol}^{-1} $, and $ \omega = 0.286 $.

In [50]:
import numpy as np
from scipy.optimize import fsolve

# 给定参数
T = 75 + 273.15  # K
P = 15  # bar
R = 83.14  # cm³·bar/(mol·K)

# 六氟化硫的临界参数
Tc = 318.7  # K
Pc = 37.6   # bar
Vc = 198    # cm³/mol
ω = 0.286

print(f"温度: {T:.2f} K")
print(f"压力: {P} bar")
print(f"对比温度 Tr = {T/Tc:.4f}")
print(f"对比压力 Pr = {P/Pc:.4f}")

温度: 348.15 K
压力: 15 bar
对比温度 Tr = 1.0924
对比压力 Pr = 0.3989


(a) The truncated virial equation $ Z = \frac{pV}{RT} = 1 + \frac{B}{V} + \frac{C}{V^2} $ with the following experimental values of virial coefficients:
$ B = -194 \, \text{cm}^3 \cdot \text{mol}^{-1} $, $ C = 15,300 \, \text{cm}^6 \cdot \text{mol}^{-2} $

In [51]:
def virial_eq_a(V):
    B = -194  # cm³/mol
    C = 15300  # cm⁶/mol²
    return P*V/(R*T) - (1 + B/V + C/V**2)

# 求解
V_initial = R*T/P  # 理想气体初始估计
V_a = fsolve(virial_eq_a, V_initial)[0]
Z_a = P*V_a/(R*T)

print(f"\n(a) 截断维里方程 (B,C给定):")
print(f"体积 V = {V_a:.2f} cm³/mol")
print(f"压缩因子 Z = {Z_a:.4f}")


(a) 截断维里方程 (B,C给定):
体积 V = 1722.27 cm³/mol
压缩因子 Z = 0.8925


(b) The truncated virial equation $ Z = \frac{pV}{RT} = 1 + \frac{Bp}{RT} $, with a value of $ B $ from the generalized Pitzer correlation.

In [52]:
def virial_eq_b():
    Tr = T/Tc
    # Pitzer关联式
    B0 = 0.083 - 0.422/(Tr**1.6)
    B1 = 0.139 - 0.172/(Tr**4.2)
    
    # 第二维里系数
    B = R*Tc/Pc * (B0 + ω*B1)  # cm³/mol
    
    # 直接计算Z
    Z_b = 1 + B*P/(R*T)
    V_b = Z_b * R * T / P
    
    return Z_b, V_b, B

Z_b, V_b, B_pitzer = virial_eq_b()
print(f"\n(b) 截断维里方程 (Pitzer关联式):")
print(f"B0 = {0.083 - 0.422/((T/Tc)**1.6):.4f}")
print(f"B1 = {0.139 - 0.172/((T/Tc)**4.2):.4f}")
print(f"第二维里系数 B = {B_pitzer:.2f} cm³/mol")
print(f"体积 V = {V_b:.2f} cm³/mol")
print(f"压缩因子 Z = {Z_b:.4f}")


(b) 截断维里方程 (Pitzer关联式):
B0 = -0.2834
B1 = 0.0203
第二维里系数 B = -195.58 cm³/mol
体积 V = 1734.10 cm³/mol
压缩因子 Z = 0.8986


(c) The Redlich/Kwong equation

In [53]:
def R_K_eq(V):
    a = 0.42748 * R**2 * Tc**2.5 / Pc
    b = 0.08664 * R * Tc / Pc
    
    return P - R*T/(V-b) + a/(T**0.5 * V * (V+b))

V_initial = R*T/P
V_c = fsolve(R_K_eq, V_initial)[0]
Z_c = P*V_c/(R*T)

print(f"\n(c) RK方程:")
print(f"参数 a = {0.42748 * R**2 * Tc**2.5 / Pc:.2f}")
print(f"参数 b = {0.08664 * R * Tc / Pc:.2f} cm³/mol")
print(f"体积 V = {V_c:.2f} cm³/mol")
print(f"压缩因子 Z = {Z_c:.4f}")


(c) RK方程:
参数 a = 142496157.94
参数 b = 61.06 cm³/mol
体积 V = 1714.15 cm³/mol
压缩因子 Z = 0.8883


(d) The Soave/Redlich/Kwong equation

In [54]:
def SRK_eq(V):
    Tr = T/Tc
    m = 0.480 + 1.574*ω - 0.176*ω**2
    α = (1 + m*(1 - Tr**0.5))**2
    
    a = 0.42748 * R**2 * Tc**2 / Pc * α
    b = 0.08664 * R * Tc / Pc
    
    return P - R*T/(V-b) + a/(V * (V+b))

V_initial = R*T/P
V_d = fsolve(SRK_eq, V_initial)[0]
Z_d = P*V_d/(R*T)

print(f"\n(d) 索阿韦-雷德利希-邝方程:")
print(f"参数 m = {0.480 + 1.574*ω - 0.176*ω**2:.4f}")
print(f"参数 α = {(1 + (0.480 + 1.574*ω - 0.176*ω**2)*(1 - (T/Tc)**0.5))**2:.4f}")
print(f"参数 a = {0.42748 * R**2 * Tc**2 / Pc * (1 + (0.480 + 1.574*ω - 0.176*ω**2)*(1 - (T/Tc)**0.5))**2:.2f}")
print(f"参数 b = {0.08664 * R * Tc / Pc:.2f} cm³/mol")
print(f"体积 V = {V_d:.2f} cm³/mol")
print(f"压缩因子 Z = {Z_d:.4f}")


(d) 索阿韦-雷德利希-邝方程:
参数 m = 0.9158
参数 α = 0.9190
参数 a = 7335133.50
参数 b = 61.06 cm³/mol
体积 V = 1726.90 cm³/mol
压缩因子 Z = 0.8949


(e) The Peng/Robinson equation

In [55]:
def PR_eq(V):
    Tr = T/Tc
    k = 0.37646 + 1.54226*ω - 0.26992*ω**2
    α = (1 + k*(1 - Tr**0.5))**2
    
    a = 0.45724 * R**2 * Tc**2 / Pc * α
    b = 0.07780 * R * Tc / Pc
    
    return P - R*T/(V-b) + a/(V*(V+b) + b*(V-b))

V_initial = R*T/P
V_e = fsolve(PR_eq, V_initial)[0]
Z_e = P*V_e/(R*T)

print(f"\n(e) 彭-罗宾逊方程:")
print(f"参数 k = {0.37464 + 1.54226*ω - 0.26992*ω**2:.4f}")
print(f"参数 α = {(1 + (0.37464 + 1.54226*ω - 0.26992*ω**2)*(1 - (T/Tc)**0.5))**2:.4f}")
print(f"参数 a = {0.45724 * R**2 * Tc**2 / Pc * (1 + (0.37464 + 1.54226*ω - 0.26992*ω**2)*(1 - (T/Tc)**0.5))**2:.2f}")
print(f"参数 b = {0.07780 * R * Tc / Pc:.2f} cm³/mol")
print(f"体积 V = {V_e:.2f} cm³/mol")
print(f"压缩因子 Z = {Z_e:.4f}")


(e) 彭-罗宾逊方程:
参数 k = 0.7936
参数 α = 0.9296
参数 a = 7936363.94
参数 b = 54.83 cm³/mol
体积 V = 1701.57 cm³/mol
压缩因子 Z = 0.8818


汇总结果

In [56]:
# 汇总结果
print(f"{'方法':<25} {'体积 (cm³/mol)':<15} {'压缩因子 Z':<10}")
print("-" * 50)
print(f"{'(a) 维里方程(B,C给定)':<25} {V_a:<15.2f} {Z_a:<10.4f}")
print(f"{'(b) 维里方程(Pitzer)':<25} {V_b:<15.2f} {Z_b:<10.4f}")
print(f"{'(c) R-K方程':<25} {V_c:<15.2f} {Z_c:<10.4f}")
print(f"{'(d) SRK方程':<25} {V_d:<15.2f} {Z_d:<10.4f}")
print(f"{'(e) PR方程':<25} {V_e:<15.2f} {Z_e:<10.4f}")
print(f"{'理想气体':<25} {R*T/P:<15.2f} {1.0000:<10.4f}")

方法                        体积 (cm³/mol)    压缩因子 Z    
--------------------------------------------------
(a) 维里方程(B,C给定)           1722.27         0.8925    
(b) 维里方程(Pitzer)          1734.10         0.8986    
(c) R-K方程                 1714.15         0.8883    
(d) SRK方程                 1726.90         0.8949    
(e) PR方程                  1701.57         0.8818    
理想气体                      1929.68         1.0000    


AI使用说明：相关公式手动搜集自PPT，交由AI生成代码并经过人工核对